# Preprocessing topics

In [10]:
import sys
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
from topics import Turn, load_turns_from_ikat_topic_files, save_turns_to_json, load_turns_from_json,Result,Reformulation
from dataclasses import asdict
import json

# downloaded from ikat website, in ikat format
ikat_topic_file = "/data/rech/huiyuche/TREC_iKAT_2024/data/topics/2023_ikat_test_topics.json"


#where_to_save = "/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat_2023_test.json"

# where you want to put the processed file.
where_to_save = "/data/rech/huiyuche/TREC_iKAT_2024/test/ikat_2023_test.json"

Then use the following code to preprocess trec topics.

In [11]:
list_of_turns = load_turns_from_ikat_topic_files(ikat_topic_file)
turn_dict_list = save_turns_to_json(list_of_turns, where_to_save)
print(json.dumps(turn_dict_list[0], indent=4))

{
    "turn_id": "9-1-1",
    "conversation_id": "9-1",
    "title": "Finding a diet",
    "current_utterance": "Can you help me find a diet for myself?",
    "current_response": "Sure, these diets fit your condition and preference: 1) Vegan Mediterranean combines the principles of the Mediterranean diet with veganism and emphasizes whole grains, vegetables, beans, nuts, seeds, herbs, and spices. It excludes animal foods while incorporating plant-based alternatives, offering a sustainable and compassionate approach. (2) Vegan Keto: is a high-fat, low-carbohydrate, and moderate-protein diet which is also exclusively plant-based. It suits individuals following a vegan lifestyle who wish to adopt a ketogenic eating plan. (3) Eco-Atkins is a vegetarian variation of the Atkins diet which focuses on whole grains, vegetables, tofu, legumes, nuts, and seeds while minimizing animal products. By reducing carbohydrate intake and incorporating plant-based protein sources, it promotes sustainabilit

# Preprocessing clueweb collection
use `/data_preprocessing_scripts/convert_tsv_to_pyserini_file.py` 
to preprocess .tsv file and convert to jsonl file.
# Preprocessing qrel
use `/data_preprocessing_scripts/preprocess_qrel.py` to preprocess qrel file downloaded from ikat website. (just replace _ with -)



# indexing
#### BM25
use `/src/indexing/jsonl_sparse_indexing.sh` to index the preprocessed .jsonl file.
#### Splade

# Evaluation
#### Scripts to use
To run the retrieval + reranking + generation pipeline, you should run `/src/evaluate/run_ikat_23_bm25_rankgpt_evaluation.sh`. The parameters are:
- `--collection` : the name of the collection to use. This argument will be used to generate the file name of evaluation result files. Default is `ClueWeb_ikat`. Can be `[ClueWeb22B_ikat]`.
- `--topics` : Name of test queries. This argument will be used to generate the file name of evaluation result files. Default is `ikat_23_test`. Can be `[ikat_23_test, ikat_24_test]`.
- `--input_query_path` : the path to the input query file. Default is `../../data/topics/ikat_2023_test.json`.
- `--index_dir_path` : the path to the directory containing the index files. Default is `../../data/indexes/clueweb22b_ikat23_fengran_sparse_index_2`.
- `--output_dir_path` : the path to the directory where the output files will be saved. Default is `../../results`.
- `--qrel_file_path` : the path to the QREL file. This file contains the ground truth relevance judgments. Default is `../../data/qrels/ikat_23_qrel.txt`.
- `--retrieval_model` : the retrieval model to use. Default is `BM25`. Can be `[BM25, ance, dpr, splade]`.
- `--cache_dir` : the path to the cache directory. Default is `/data/rech/huiyuche/huggingface`. Cache directory for Huggingface models.
- `--dense_query_encoder_path` : the path to the dense query encoder model. Default is `castorini/ance-msmarco-passage`. Should be a Huggingface format folder/link to a model.
- `--reranker` : the reranker model to use. Default is `none`. Can be `['none', rankllama, rankgpt, 'monot5_base', 'monot5_base_10k']`.
- `--rerank_quant` : the quantization method for reranking. Default is `none`. Can be `['none', '8b', '4b']`.
- `--rankgpt_llm` : the language model to use for RankGPT. Default is `gpt-3.5-turbo`. Can be `['gpt-3.5-turbo']`.
- `--window_size` : the size of the window for RankGPT. Default is `5`.
- `--step` : the step size for RankGPT. Default is `1`.
- `--generation_model` : model for response generation.
- `--bm25_k1` : the `k1` parameter for the BM25 model. Default is `0.9`.
- `--bm25_b` : the `b` parameter for the BM25 model. Default is `0.4`.
- `--use_rm3` : if RM3 will be used. Default is `False`.
- `--fb_terms` : the number of feedback terms for the RM3 model. Default is `10`.
- `--fb_docs` : the number of feedback documents for the RM3 model. Default is `10`.
- `--original_query_weight` : the weight of the original query in the RM3 model. Default is `0.5`.
- `--retrieval_top_k` : the number of top documents to retrieve. Default is `1000`.
- `--rerank_top_k` : the number of top documents to rerank. Default is `50`.
- `--generation_top_k` : the number of top documents to generate responses from. Default is `3`.
- `--metrics` : the evaluation metrics to compute. Default is `map, ndcg_cut.1, ndcg_cut.3, ndcg_cut.5, ndcg_cut.10, P.1, P.3, P.5, P.10, recall.5, recall.100, recall.1000, recip_rank`. Should be a comma-separated string of metrics.
- `--save_metrics_to_object` : whether to save metrics to an object. Default is `False`.




#### Where to find results
For each run, the results are automatically saved to /results/args.collection/args.topics. There are three sub-folders:
1. metrics. Contains:
    1. for each metric, a .txt file containing the metric value accross all runs.
    2. for each run, a . json file containing all the metrics calculated in the run.
2. per_qeury_metrics: for each run, a .json file containing all the metrics calculated for each query.
3. ranking: for each run, a .txt file containing the trec format ranking result. 